# La música en la Catedral de Santo Domingo de la Calzada
## Aplicación para la búsqueda de palabras del Capítulo 1

In [1]:
import numpy as np
import cv2
import sys
import pickle
import re,string
from unicodedata import normalize
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [4]:
!wget https://download1518.mediafire.com/tvwo4ks4dfzg/0mdhm03kaxw7vtj/Fotos.zip

--2020-11-26 09:25:48--  https://download1518.mediafire.com/tvwo4ks4dfzg/0mdhm03kaxw7vtj/Fotos.zip
Resolving download1518.mediafire.com (download1518.mediafire.com)... 205.196.123.206
Connecting to download1518.mediafire.com (download1518.mediafire.com)|205.196.123.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6707373 (6.4M) [application/zip]
Saving to: ‘Fotos.zip’

Fotos.zip           100%[===================>]   6.40M  13.9MB/s    in 0.5s    

2020-11-26 09:25:48 (13.9 MB/s) - ‘Fotos.zip’ saved [6707373/6707373]



In [5]:
!unzip Fotos.zip

Archive:  Fotos.zip
  inflating: Fotos/pag1.jpg          
  inflating: Fotos/pag10.jpg         
  inflating: Fotos/pag11.jpg         
  inflating: Fotos/pag12.jpg         
  inflating: Fotos/pag13.jpg         
  inflating: Fotos/pag14.jpg         
  inflating: Fotos/pag15.jpg         
  inflating: Fotos/pag16.jpg         
  inflating: Fotos/pag17.jpg         
  inflating: Fotos/pag18.jpg         
  inflating: Fotos/pag19.jpg         
  inflating: Fotos/pag2.jpg          
  inflating: Fotos/pag20.jpg         
  inflating: Fotos/pag21.jpg         
  inflating: Fotos/pag22.jpg         
  inflating: Fotos/pag23.jpg         
  inflating: Fotos/pag3.jpg          
  inflating: Fotos/pag4.jpg          
  inflating: Fotos/pag5.jpg          
  inflating: Fotos/pag6.jpg          
  inflating: Fotos/pag7.jpg          
  inflating: Fotos/pag8.jpg          
  inflating: Fotos/pag9.jpg          
  inflating: texto.pickle            


In [6]:
with open("texto.pickle", "rb") as f:
    texto = pickle.load(f)

In [7]:
def remove_punctuation(frase):
  char = re.sub('[%s]' % re.escape(string.punctuation.replace('-','')),'',frase)
  char = char.replace('¿','').replace('?','').replace('`','').replace('´','').replace('#','').lower()
  char = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", char), 0, re.I
    )
  char = normalize( 'NFC', char)
  return char

In [8]:
def buscar(frase,texto): # Las fotos están en una carpeta. El texto es nuestro diccionario con la información.
  frase2 = frase.upper()
  fotos = []
  path = '/content/Fotos/' # Las fotos están almacenadas en una carpeta, que más tarde será una base de datos.
  frase = frase.split()
  n = len(frase)
  frase = ''.join(frase)
  frase =  remove_punctuation(frase).replace(' ','')
  for cla in texto:
    foto = cv2.imread(path + 'pag' + str(cla) + '.jpg') # Vamos abriendo las fotos página a página
    foto = cv2.cvtColor(foto, cv2.COLOR_BGR2RGB)
    N = len(texto[cla]['text'])
    j = 0
    encontrado = False
    while j < N:
      k = j
      b = True
      siguiente = texto[cla]['text'][j:n+j]
      if siguiente[-1][-1] == '-':
        siguiente = ''.join(texto[cla]['text'][j:n+j+1])
        siguiente = siguiente.replace(' ','').replace('-','')
        j = j + 1
      else:
        siguiente = ''.join(siguiente)
        j = j + 1
        b = False

      if frase in siguiente:
        encontrado = True
        x = texto[cla]['left'][k]
        y = texto[cla]['top'][k]
        h = texto[cla]['height'][k]
        w = 0
        for i in range(k,k+n):
          w = w + texto[cla]['width'][i] + 7 # le añado un poco más para tener en cuenta la separación entre palabras 
        cv2.rectangle(foto,(x-10,y-10),(x+w+10,y+h+10),(226,182,19),2)
        if b:
          x = texto[cla]['left'][k+n]
          y = texto[cla]['top'][k+n]
          h = texto[cla]['height'][k+n]
          w = texto[cla]['width'][k+n]
          cv2.rectangle(foto,(x-10,y-10),(x+w+10,y+h+10),(226,182,19),2)
    if encontrado == True: fotos.append(foto)

  n = len(fotos)
  if n == 0:
    print('La palabra no aparece o está mal escrita.')
  else:
    if len(fotos) == 1:
      print('LA PALABRA "' + frase2 + '" APARECE EN ' + str(1) + ' PÁGINA.')
    else:
      print('LA PALABRA "' + frase2 + '" APARECE EN ' + str(len(fotos)) + ' PÁGINA.')
    fig, ax = plt.subplots(n,1,figsize=(16*n,16*n))
    for i in range(0,n):
      if n == 1:
        ax.imshow(fotos[i])
      else:
        ax[i].imshow(fotos[i])
    plt.show()

In [9]:
keyword=widgets.Text(
    placeholder='Escribe algo',
    disabled=False
)
display(keyword)


button = widgets.Button(description="Buscar")
display(button)

output = widgets.Output()
@output.capture()
def on_button_clicked(b):
  output.clear_output()
  if keyword.value == '': 
    print('No has escrito nada.')
  else:
    buscar(keyword.value,texto)

button.on_click(on_button_clicked)
display(output)

Text(value='', placeholder='Escribe algo')

Button(description='Buscar', style=ButtonStyle())

Output()